# Imports

In [1]:
import os, sys, time, glob, random, argparse
import wandb
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import time
import tqdm
import scipy.stats as stats
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

# API
from nats_bench import create

# custom modules
from custom.tss_model import TinyNetwork
from xautodl.models.cell_searchs.genotypes import Structure
from ZeroShotProxy import *
from tss_utils import compute_vkdnw, plot_stats, get_stats, get_metrics, analyze_results, generate_accs, get_results_from_api, get_scores

# All

In [2]:
# scp 'tyblondr@login.rci.cvut.cz:~/PycharmProjects/VKDNW/NB201/*_log.csv' data_0811/

target = 'val_accs'
dataset = 'ImageNet16-120'
compute_graf = True
zero_cost_score_list = ['vkdnw', 'vkdnw_dim', 'vkdnw_chisquare', 'az_nas', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'te_nas', 'flops', 'trainability', 'expressivity', 'progressivity']
plt.rcParams['text.usetex'] = False
compute_graf_str = 'filtered' if compute_graf else 'unfiltered'

In [3]:
run_dict = {
    'cifar10': {
        # same images
        #'nazderaze/VKDNW/n6n44keg': 1,
        #'nazderaze/VKDNW/6c1pv095': 2,
        #'nazderaze/VKDNW/45us11be': 3,
        #'nazderaze/VKDNW/rwp2qvkw': 4,
        #'nazderaze/VKDNW/6bsssh4z': 5,
        
        # different images
        #'nazderaze/VKDNW/7a8jm975' : -1 # testing (successfull)
        #'nazderaze/VKDNW/cixw6r9b': 1,
        #'nazderaze/VKDNW/1nikalf3': 2,
        #'nazderaze/VKDNW/r5l3qg12': 3,
        #'nazderaze/VKDNW/fq581h0d': 4,
        #'nazderaze/VKDNW/0yfzaxra': 5,
        
        # entropy
        #'nazderaze/VKDNW/87vy5rby': 1,
        #'nazderaze/VKDNW/8sd2hdws': 5,
        #'nazderaze/VKDNW/bh09isa0': 3,
        #'nazderaze/VKDNW/ei31s875': 4,
        #'nazderaze/VKDNW/koxirqt3': 2,
        
        # entropy all
        #'nazderaze/VKDNW/6lcy0jqp': 1,
        #'nazderaze/VKDNW/orl2me1l': 2,
        #'nazderaze/VKDNW/y5n4lnsn': 3,
        #'...': 4,
        #'nazderaze/VKDNW/7gkdl3gl': 5,
        
        # final
        'nazderaze/VKDNW/k7llaf2s': 1,
        'nazderaze/VKDNW/knu2gv65': 2,
        'nazderaze/VKDNW/fnfbpspk': 3,
        'nazderaze/VKDNW/w4b39d0d': 4,
        'nazderaze/VKDNW/vbr5n8iy': 5
        
    },
    'cifar100': {
        # same images
        #'nazderaze/VKDNW/qmq5vp3k': 1,
        #'nazderaze/VKDNW/31lrq6p7': 2,
        #'nazderaze/VKDNW/3qazc6po': 3,
        #'nazderaze/VKDNW/424twoyv': 4,
        #'nazderaze/VKDNW/783h4opf': 5,
        
        # different images
        #'nazderaze/VKDNW/n2m8i53l': 1,
        #'nazderaze/VKDNW/48xbnfdg': 2,
        #'nazderaze/VKDNW/f0czkx5u': 3,
        #'nazderaze/VKDNW/ldv3b1bh': 4,
        #'nazderaze/VKDNW/qwae4nqx': 5,
        
        # entropy
        #'nazderaze/VKDNW/lfnin2ui': 1,
        #'nazderaze/VKDNW/1cs8orlm': 4,
        #'nazderaze/VKDNW/4tep5lv0': 5,
        #'nazderaze/VKDNW/lqka0nwl': 2,
        #'nazderaze/VKDNW/mqrfyv7t': 3,
        
        # entropy all
        #'...': 1,
        #'...': 2,
        #'...': 3,
        #'...': 4,
        #'...': 5,
        
        # final
        'nazderaze/VKDNW/v2b816ul': 1,
        'nazderaze/VKDNW/velwxtxd': 2,
        'nazderaze/VKDNW/f3ljf5jf': 3,
        'nazderaze/VKDNW/e3w3dkv7': 4,
        'nazderaze/VKDNW/9ibvj04q': 5,
    },
    'ImageNet16-120': {
        # same images
        #'nazderaze/VKDNW/ftg0tdsa': 1,
        #'nazderaze/VKDNW/vqf1ey6x': 2,
        #'nazderaze/VKDNW/v0a0m67q': 3,
        #'nazderaze/VKDNW/uiv37u18': 4,
        #'nazderaze/VKDNW/c1338vfg': 5,
        
        # different images
        #'nazderaze/VKDNW/55f1omxn': 1,
        #'nazderaze/VKDNW/amdcxrz7': 2,
        #'nazderaze/VKDNW/sl0rjhwh': 3,
        #'nazderaze/VKDNW/z2ph6iav': 4,
        #'nazderaze/VKDNW/ol9rwkeo': 5,
        
        # entropy
        # 'nazderaze/VKDNW/hwtw58ot': -1 # test
        #'nazderaze/VKDNW/u214u8p1': 4,
        #'nazderaze/VKDNW/bycu6ed3': 5,
        #'nazderaze/VKDNW/2ztcsos6': 3,
        #'nazderaze/VKDNW/kptcxbi2': 2,
        #'nazderaze/VKDNW/f9n7j83e': 1,
        
        #'nazderaze/VKDNW/hrrxcexh': 1,
        #'nazderaze/VKDNW/jwhy017w': 2,
        #'nazderaze/VKDNW/82rycftz': 3,
        #'nazderaze/VKDNW/bw6b93p9': 4,
        #'nazderaze/VKDNW/ca423k7t': 5,
        
        # final
        'nazderaze/VKDNW/ss2kwvpp': 1,
        'nazderaze/VKDNW/es9t2696': 2,
        'nazderaze/VKDNW/m1dusbs9': 3,
        'nazderaze/VKDNW/kgmcpu2y': 4,
        'nazderaze/VKDNW/cpbmsvuw': 5,
        
    }
}

In [4]:
api_nats = create('/mnt/personal/tyblondr/NATS-tss-v1_0-3ffb9-simple/', 'tss', fast_mode=True, verbose=False)

if os.path.exists(f"./tss_features_{dataset}.pickle"):
    archs = pd.read_pickle(f"./tss_features_{dataset}.pickle")
else:
    archs = generate_accs(api_nats, dataset=dataset)
    print(f'No. of generated archs: {archs.shape[0]}')
    archs.to_pickle(f"./tss_features_{dataset}.pickle")

In [5]:
api_wandb = wandb.Api()

log = None
results = None
for run_id, seed in run_dict[dataset].items():
    
    run = pd.DataFrame(api_wandb.run(run_id).scan_history())
    run.rename({'arch': 'net_str'}, axis=1, inplace=True)
    
    run = pd.merge(archs, run, on='net_str', how='inner')
    if compute_graf:
        run = run.loc[run['net'].notnull(), :]  # keep only nets with features
        
    print(f'No. of archs for seed {seed} after filtering: {run.shape[0]}.')
    
    for col in run.columns:
        if col not in ['net_str', 'net']:
            run[col] = run[col].astype(float)
    
    if 'jacov' in run.columns:        
        run['jacov'] = run['jacov'].fillna(run['jacov'].min()).astype(float)

    df_scores = get_scores(run.copy(), compute_graf=compute_graf, zero_cost_score_list=zero_cost_score_list)
    df_scores['dataset'] = dataset
    df_scores['seed'] = seed
    if results is None:
        results = df_scores
    else:
        results = pd.concat([results, df_scores], ignore_index=True)
print(f'Total number of records: {results.shape[0]}')

No. of archs for seed 1 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running expressivity
Running progressivity
No. of archs for seed 2 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running expressivity
Running progressivity
No. of archs for seed 3 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running expres

In [6]:
log = None
for seed in results['seed'].unique():
    
    results_temp = results.loc[results['seed'] == seed, :].copy()
    for zero_cost_rank in [p for p in results_temp.columns if '_rank' in p]:
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.nan, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
        log_temp = pd.DataFrame(get_metrics(results_temp, pred_name=zero_cost_rank, show_plot=False, seed=seed))
        
        if log is None:
            log = log_temp.copy()
        else:
            log = pd.concat([log, log_temp.copy()], ignore_index=True)
            
        if seed == min(results['seed'].unique()):
            plot_stats(get_stats(results_temp, 'vkdnw_dim', target, zero_cost_rank), 'vkdnw_dim', target, zero_cost_rank, f'{dataset}_{str(compute_graf)}_{zero_cost_rank}')

log = log.groupby('pred_name', as_index=False).agg(['mean', 'std']).reset_index()
log['dataset'] = dataset
log['no_seeds'] = len(results['seed'].unique())
log['archs_filtered'] = compute_graf_str
log

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

pred_name   kendall            spearman             pearson  \
                              mean       std      mean       std      mean   
0            az_nas_rank  0.672816  0.001249  0.858689  0.001364  0.813306   
1      expressivity_rank  0.586514  0.000726  0.777940  0.000614  0.719885   
2             flops_rank  0.508642  0.000000  0.700391  0.000000  0.667450   
3         grad_norm_rank  0.310441  0.000504  0.417613  0.000802  0.394772   
4          gradsign_rank -0.454410  0.002430 -0.626828  0.003195 -0.602359   
5             grasp_rank  0.359110  0.003922  0.501660  0.006311  0.485504   
6             jacov_rank  0.602018  0.001780  0.779219  0.001888  0.751795   
7            naswot_rank  0.605221  0.000404  0.794006  0.000386  0.760380   
8     progressivity_rank  0.458143  0.004652  0.629605  0.005400  0.612273   
9              snip_rank  0.389260  0.000437  0.521150  0.000602  0.511715   
10          synflow_rank  0.531145  0.000146  0.718543  0.000091  0.686889   
11           te_nas_rank  0.522621  0.001744  0.709051  0.001982  0.669406   
12     trainability_rank  0.352995  0.000404  0.517155  0.000440  0.499147   
13  vkdnw_chisquare_rank  0.149833  0.003743  0.228090  0.004528  0.297678   
14       vkdnw_comb_rank  0.742428  0.000472  0.905600  0.000335  0.848436   
15        vkdnw_dim_rank  0.512044  0.000000  0.706719  0.000000  0.675699   
16        vkdnw_exp_rank  0.676885  0.001133  0.863611  0.001130  0.817468   
17       vkdnw_prog_rank  0.720608  0.000346  0.895149  0.000262  0.839187   
18            vkdnw_rank  0.615831  0.000842  0.808589  0.000777  0.761127   
19      vkdnw_train_rank  0.652615  0.001421  0.832237  0.001307  0.779967   
20              zen_rank  0.091004  0.000221  0.108974  0.000380  0.134875   
21             zico_rank  0.586679  0.000583  0.778793  0.000544  0.740662   

                    ndcg50                     ndcg100  ...  ndcg1000  \
         std          mean           std          mean  ...       std   
0   0.000844  3.290842e-01  2.327173e-02  3.797528e-01  ...  0.008476   
1   0.000792  1.919631e-02  2.499150e-03  2.629433e-02  ...  0.003113   
2   0.000000  1.192523e-01  0.000000e+00  1.767912e-01  ...  0.000000   
3   0.000936  3.749862e-04  2.213798e-04  8.434647e-03  ...  0.002604   
4   0.003309  5.892313e-09  6.299651e-09  2.276518e-08  ...  0.000003   
5   0.005207  5.221175e-03  3.141932e-03  2.473023e-02  ...  0.006829   
6   0.001516  9.877489e-02  2.561853e-02  1.248206e-01  ...  0.003769   
7   0.000441  3.198674e-01  3.261148e-03  2.831819e-01  ...  0.002064   
8   0.003214  1.258578e-01  1.959717e-02  1.436977e-01  ...  0.008492   
9   0.000688  1.220818e-03  4.913652e-04  1.511454e-02  ...  0.002378   
10  0.000062  2.620345e-01  1.980444e-02  3.104594e-01  ...  0.005356   
11  0.002772  5.140519e-02  7.837536e-03  8.321281e-02  ...  0.009345   
12  0.000281  2.599841e-02  7.982936e-03  3.795140e-02  ...  0.002661   
13  0.002428  1.523658e-02  1.342166e-02  1.778754e-02  ...  0.008058   
14  0.000187  3.462363e-01  1.838347e-02  3.845662e-01  ...  0.006665   
15  0.000000  2.808418e-01  0.000000e+00  2.777116e-01  ...  0.000000   
16  0.000644  3.631038e-01  2.308826e-02  4.128562e-01  ...  0.011382   
17  0.000201  3.553547e-01  1.338885e-02  4.040283e-01  ...  0.006994   
18  0.000624  2.981178e-01  2.043445e-02  3.679977e-01  ...  0.005521   
19  0.001044  1.540287e-01  2.074403e-02  1.999624e-01  ...  0.008488   
20  0.000365  3.346791e-03  1.864885e-04  2.701769e-03  ...  0.000458   
21  0.000377  2.625693e-01  1.031410e-02  2.797526e-01  ...  0.002275   

    ndcg5000              acc_top            acc_top_true       \
        mean       std       mean        std         mean  std   
0   0.798990  0.005590  44.792222   1.361551    47.311111  0.0   
1   0.706877  0.002471  40.338889   1.702005    47.311111  0.0   
2   0.741047  0.000000  41.444444   0.000000    47.311111  0.0   
3   0.444068  0.000431  18.450000  10.127342    47.311111  

In [7]:
"""
for zero_cost_score in ['vkdnw']:
    for i in range(5):
        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)
        analyze_results(api_nats, results_temp, zero_cost_score, target)
        
df_top = None
for seed in range(1, 6):
    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()
    for pred in [p for p in results_temp.columns if '_rank' in p]:
        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]
        if df_top is None:
            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])
        else:
            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)
df_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()
"""

"\nfor zero_cost_score in ['vkdnw']:\n    for i in range(5):\n        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)\n        analyze_results(api_nats, results_temp, zero_cost_score, target)\n        \ndf_top = None\nfor seed in range(1, 6):\n    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()\n    for pred in [p for p in results_temp.columns if '_rank' in p]:\n        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]\n        if df_top is None:\n            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])\n        else:\n            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)\ndf_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()\n"

In [10]:
log_train = None
if not compute_graf:
    print('No graf prediction.')
else:
    print('Graf prediction.')
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    
    pred_lists = {
        'model_vkdnw': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_entropy', 'vkdnw_dim', 'flops'], #['vkdnw_chisquare', 'vkdnw_dim', 'flops'],
        'model_vkdnw+zs': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'],
        'model_vkdnw+zs+graf': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'] + [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p],
        'model_graf': [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p]
    }
    
    for train_size in [1024]:
        for seed in results['seed'].unique():
            
            results_temp = results.loc[results['seed'] == seed, :].copy()
            results_temp = results_temp.apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
            results_temp = results_temp.apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
            for model_name, pred_list in pred_lists.items():
                
                train_df, test_df = train_test_split(results_temp, test_size=1 - (train_size / results_temp.shape[0]), random_state=seed)
                model = Pipeline([
                    ('scaler', StandardScaler()),           # Step 1: Standardize features
                    ('regressor', RandomForestRegressor(n_estimators=100))  # Step 2: Train RandomForestRegressor
                ])
                model.fit(train_df[pred_list], train_df[target])
                test_df['pred_' + model_name] = model.predict(test_df[pred_list])
                log_train_temp = pd.DataFrame(get_metrics(test_df, 'pred_' + model_name, show_plot=False, seed=seed))        
                log_train_temp['train_size'] = train_size
            
                if log_train is None:
                    log_train = log_train_temp.copy()
                else:
                    log_train = pd.concat([log_train, log_train_temp.copy()], ignore_index=True)
                    
                if seed == min(results['seed'].unique()):
                    plot_stats(get_stats(test_df, 'vkdnw_dim', target, 'pred_' + model_name), 'vkdnw_dim', target, model_name, f'{dataset}_{str(compute_graf)}_{model_name}_{train_size}')

    log_train = log_train.groupby(['pred_name', 'train_size'], as_index=False).agg(['mean', 'std']).reset_index()
    log_train['dataset'] = dataset
    log_train['no_seeds'] = len(results['seed'].unique())
    log_train['archs_filtered'] = compute_graf_str
    log_train

Graf prediction.


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [11]:
log_train

pred_name train_size   kendall            spearman  \
                                            mean       std      mean   
0           pred_model_graf       1024  0.795205  0.002877  0.940235   
1          pred_model_vkdnw       1024  0.619734  0.002720  0.810043   
2       pred_model_vkdnw+zs       1024  0.828968  0.003924  0.957121   
3  pred_model_vkdnw+zs+graf       1024  0.839437  0.003048  0.962064   

              pearson              ndcg50            ...  ndcg1000  ndcg5000  \
        std      mean       std      mean       std  ...       std      mean   
0  0.001281  0.877776  0.002670  0.584067  0.030068  ...  0.009858  0.904959   
1  0.002414  0.761811  0.001945  0.283733  0.054741  ...  0.025775  0.798959   
2  0.001831  0.893418  0.002234  0.654465  0.074652  ...  0.015924  0.916758   
3  0.001579  0.896950  0.002088  0.664560  0.031152  ...  0.010823  0.925613   

               acc_top           acc_top_true              dataset no_seeds  \
        std       mean       std         mean  std                            
0  0.006728  46.211111  0.760482    47.311111  0.0  ImageNet16-120        5   
1  0.011298  42.844444  3.486869    47.311111  0.0  ImageNet16-120        5   
2  0.012648  45.962222  0.868467    47.311111  0.0  ImageNet16-120        5   
3  0.007419  46.384444  0.586610    47.311111  0.0  ImageNet16-120        5   

  archs_filtered  
                  
0       filtered  
1       filtered  
2       filtered  
3       filtered  

[4 rows x 23 columns]

In [12]:
if compute_graf:
    log_save = pd.concat([log, log_train], ignore_index=True)
else:
    log_save = log
    
log_save.columns = ['_'.join(col) for col in log_save.columns]
log_save.to_csv(f'{dataset}_{compute_graf_str}_log.csv', index=False)
log_save

,pred_name_,kendall_mean,kendall_std,spearman_mean,spearman_std,pearson_mean,pearson_std,ndcg50_mean,ndcg50_std,ndcg100_mean,...,ndcg5000_mean,ndcg5000_std,acc_top_mean,acc_top_std,acc_top_true_mean,acc_top_true_std,dataset_,no_seeds_,archs_filtered_,train_size_
0,az_nas_rank,0.672816,0.001249,0.858689,0.001364,0.813306,0.000844,3.290842e-01,2.327173e-02,3.797528e-01,...,0.798990,0.005590,44.792222,1.361551,47.311111,0.0,ImageNet16-120,5,filtered,NaN
1,expressivity_rank,0.586514,0.000726,0.777940,0.000614,0.719885,0.000792,1.919631e-02,2.499150e-03,2.629433e-02,...,0.706877,0.002471,40.338889,1.702005,47.311111,0.0,ImageNet16-120,5,filtered,NaN
2,flops_rank,0.508642,0.000000,0.700391,0.000000,0.667450,0.000000,1.192523e-01,0.000000e+00,1.767912e-01,...,0.741047,0.000000,41.444444,0.000000,47.311111,0.0,ImageNet16-120,5,filtered,NaN
3,grad_norm_rank,0.310441,0.000504,0.417613,0.000802,0.394772,0.000936,3.749862e-04,2.213798e-04,8.434647e-03,...,0.444068,0.000431,18.450000,10.127342,47.311111,0.0,ImageNet16-120,5,filtered,NaN
4,gradsign_rank,-0.454410,0.002430,-0.626828,0.003195,-0.602359,0.003309,5.892313e-09,6.299651e-09,2.276518e-08,...,0.059683,0.001371,12.208889,1.132227,47.311111,0.0,ImageNet16-120,5,filtered,NaN
5,grasp_rank,0.359110,0.003922,0.501660,0.006311,0.485504,0.005207,5.221175e-03,3.141932e-03,2.473023e-02,...,0.472276,0.004348,12.051111,10.259637,47.311111,0.0,ImageNet16-120,5,filtered,NaN
6,jacov_rank,0.602018,0.001780,0.779219,0.001888,0.751795,0.001516,9.877489e-02,2.561853e-02,1.248206e-01,...,0.732392,0.002704,38.788889,1.119282,47.311111,0.0,ImageNet16-120,5,filtered,NaN
7,naswot_rank,0.605221,0.000404,0.794006,0.000386,0.760380,0.000441,3.198674e-01,3.261148e-03,2.831819e-01,...,0.784324,0.000449,37.108889,3.647275,47.311111,0.0,ImageNet16-120,5,filtered,NaN
8,progressivity_rank,0.458143,0.004652,0.629605,0.005400,0.612273,0.003214,1.258578e-01,1.959717e-02,1.436977e-01,...,0.605715,0.005453,42.868889,2.220074,47.311111,0.0,ImageNet16-120,5,filtered,NaN
9,snip_rank,0.389260,0.000437,0.521150,0.000602,0.511715,0.000688,1.220818e-03,4.913652e-04,1.511454e-02,...,0.449849,0.001805,0.833333,0.000000,47.311111,0.0,ImageNet16-120,5,filtered,NaN


In [7]:
log[['pred_name', 'kendall', 'spearman', 'ndcg1000']]

pred_name   kendall            spearman            ndcg1000  \
                              mean       std      mean       std      mean   
0            az_nas_rank  0.672816  0.001249  0.858689  0.001364  0.533572   
1      expressivity_rank  0.586514  0.000726  0.777940  0.000614  0.276717   
2             flops_rank  0.508642  0.000000  0.700391  0.000000  0.397276   
3         grad_norm_rank  0.310441  0.000504  0.417613  0.000802  0.265334   
4          gradsign_rank -0.454410  0.002430 -0.626828  0.003195  0.000039   
5             grasp_rank  0.359110  0.003922  0.501660  0.006311  0.281339   
6             jacov_rank  0.602018  0.001780  0.779219  0.001888  0.356006   
7            naswot_rank  0.605221  0.000404  0.794006  0.000386  0.490074   
8     progressivity_rank  0.458143  0.004652  0.629605  0.005400  0.336650   
9              snip_rank  0.389260  0.000437  0.521150  0.000602  0.285671   
10          synflow_rank  0.531145  0.000146  0.718543  0.000091  0.510760   
11           te_nas_rank  0.522621  0.001744  0.709051  0.001982  0.330389   
12     trainability_rank  0.352995  0.000404  0.517155  0.000440  0.233822   
13  vkdnw_chisquare_rank  0.149833  0.003743  0.228090  0.004528  0.065190   
14       vkdnw_comb_rank  0.742428  0.000472  0.905600  0.000335  0.670701   
15        vkdnw_dim_rank  0.512044  0.000000  0.706719  0.000000  0.477064   
16        vkdnw_exp_rank  0.676885  0.001133  0.863611  0.001130  0.574496   
17       vkdnw_prog_rank  0.720608  0.000346  0.895149  0.000262  0.655250   
18            vkdnw_rank  0.615831  0.000842  0.808589  0.000777  0.594699   
19      vkdnw_train_rank  0.652615  0.001421  0.832237  0.001307  0.452213   
20              zen_rank  0.091004  0.000221  0.108974  0.000380  0.072511   
21             zico_rank  0.586679  0.000583  0.778793  0.000544  0.523170   

              
         std  
0   0.008476  
1   0.003113  
2   0.000000  
3   0.002604  
4   0.000003  
5   0.006829  
6   0.003769  
7   0.002064  
8   0.008492  
9   0.002378  
10  0.005356  
11  0.009345  
12  0.002661  
13  0.008058  
14  0.006665  
15  0.000000  
16  0.011382  
17  0.006994  
18  0.005521  
19  0.008488  
20  0.000458  
21  0.002275

In [9]:
log[['pred_name', 'ndcg1000']]

pred_name  ndcg1000          
                              mean       std
0            az_nas_rank  0.533572  0.008476
1      expressivity_rank  0.276717  0.003113
2             flops_rank  0.402924  0.000000
3         grad_norm_rank  0.265334  0.002604
4          gradsign_rank  0.000039  0.000003
5             grasp_rank  0.281339  0.006829
6             jacov_rank  0.356006  0.003769
7            naswot_rank  0.490074  0.002064
8     progressivity_rank  0.336650  0.008492
9              snip_rank  0.285671  0.002378
10          synflow_rank  0.510760  0.005356
11           te_nas_rank  0.330389  0.009345
12     trainability_rank  0.233822  0.002661
13  vkdnw_chisquare_rank  0.065191  0.008057
14       vkdnw_comb_rank  0.670701  0.006665
15        vkdnw_dim_rank  0.497171  0.000000
16        vkdnw_exp_rank  0.574496  0.011382
17       vkdnw_prog_rank  0.655250  0.006994
18            vkdnw_rank  0.594699  0.005521
19      vkdnw_train_rank  0.452213  0.008488
20              zen_rank  0.072511  0.000458
21             zico_rank  0.523170  0.002275